In [3]:
import boto3
from botocore.client import Config
import os

class S3Storage:
    def __init__(self):
        self.S3_ACCESS_KEY = os.environ.get("S3_ACCESS_KEY", "")
        self.S3_SECRET_KEY = os.environ.get("S3_SECRET_KEY", "")
        self.S3_ENDPOINT = os.environ.get("S3_ENDPOINT", "")
        self.S3_REGION = os.environ.get("S3_REGION", "")
        self.bucket_name = os.environ.get("S3_BUCKET", "")
        self.bucket_name = "science-infuse-content"
        
        self.s3_client = boto3.client(
            "s3",
            aws_access_key_id=self.S3_ACCESS_KEY,
            aws_secret_access_key=self.S3_SECRET_KEY,
            endpoint_url=self.S3_ENDPOINT,
            region_name=self.S3_REGION
        )

    def list_buckets(self):
        response = self.s3_client.list_buckets()
        print("Buckets existants :")
        for bucket in response['Buckets']:
            print(f'  {bucket["Name"]}')

    def list_objects(self):
        response = self.s3_client.list_objects_v2(Bucket=self.bucket_name)
        # print(f"Objets dans le bucket {self.bucket_name} :")
        return [obj['Key'] for obj in response.get('Contents', [])]
        # for obj in response.get('Contents', []):
        #     print(f'  {obj["Key"]}')

    def upload_file(self, file_path, object_name):
        try:
            self.s3_client.upload_file(file_path, self.bucket_name, object_name, ExtraArgs={'ACL': 'public-read'})
            print(f"File {file_path} uploaded successfully to bucket {self.bucket_name} as {object_name}.")
            
            # Generate the public URL
            public_url = f"{self.S3_ENDPOINT}/{self.bucket_name}/{object_name}"
            print(f"Public URL: {public_url}")
            return public_url
        except Exception as e:
            print(f"Error during file upload: {e}")
            return None

    def get_presigned_url(self, object_name, expiration=3600):
        return self.s3_client.generate_presigned_url(
            ClientMethod='get_object',
            Params={'Bucket': self.bucket_name, 'Key': object_name},
            ExpiresIn=expiration  # URL expiration time in seconds
        )
    
    def remove_file(self, object_name):
        try:
            self.s3_client.delete_object(Bucket=self.bucket_name, Key=object_name)
            print(f"File {object_name} successfully removed from bucket {self.bucket_name}.")
            return True
        except Exception as e:
            print(f"Error during file removal: {e}")
            return False


In [2]:
import sys
import dotenv

print(dotenv.load_dotenv('../../.env'))

import os
print(os.environ.get("S3_BUCKET"))
s3 = S3Storage()

# s3.list_buckets()
# s3.get_presigned_url("pdf/40776654-f565-44c5-b0b3-a5b1f1d74636.pdf", expiration=1000)

# for key in s3.list_objects():
#     print(key)
print(len([obj for obj in s3.list_objects() if "youtube" in obj]))
# file_path = './documents/youtube/40696c1b-6561-49ca-bb49-d20990d87040.mp4'
# object_name = '/documents/youtube/40696c1b-6561-49ca-bb49-d20990d87040.mp4'
# public_url = s3.upload_file(file_path, object_name)

# if public_url:
#     presigned_url = s3.get_presigned_url(object_name)
#     print(f"Presigned URL: {presigned_url}")


True
science-infuse-content-dev


NameError: name 'S3Storage' is not defined

In [5]:
import boto3
import json

import sys
import dotenv

print(dotenv.load_dotenv('../../.env'))

s3 = S3Storage()


bucket_name = "science-infuse-content"
cors_configuration = {
    'CORSRules': [{
        'AllowedHeaders': ['*'],
        'AllowedMethods': ['GET', 'HEAD'],
        'AllowedOrigins': ['http://51.38.223.168', 'https://51.38.223.168', 'http://localhost:3000'],
        'MaxAgeSeconds': 3000
    }]
}

s3.s3_client.put_bucket_cors(Bucket=s3.bucket_name,
                   CORSConfiguration=cors_configuration)

True


{'ResponseMetadata': {'RequestId': 'tx2cac79fafcf94686b677b-0066d17ea9',
  'HostId': 'tx2cac79fafcf94686b677b-0066d17ea9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'text/html; charset=UTF-8',
   'x-amz-id-2': 'tx2cac79fafcf94686b677b-0066d17ea9',
   'x-amz-request-id': 'tx2cac79fafcf94686b677b-0066d17ea9',
   'content-length': '0',
   'date': 'Fri, 30 Aug 2024 08:11:21 GMT'},
  'RetryAttempts': 0}}